<a href="https://colab.research.google.com/github/MayerT1/Pakistan_Research/blob/main/DL_Training_Chips.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://geemap.org/notebooks/96_image_chips/#create-a-function-for-downloading-image

In [20]:
from google.colab import auth
auth.authenticate_user()

In [23]:
!pwd

/content/drive/MyDrive/PhD_Main_Folder/Methods/Image_Chips


In [22]:
%cd /content/drive/MyDrive/PhD_Main_Folder/Methods/Image_Chips

/content/drive/MyDrive/PhD_Main_Folder/Methods/Image_Chips


In [3]:
!pip install geemap

In [4]:
import ee
import geemap
import geemap.chart as chart

In [5]:
import ee
import geemap
import logging
import multiprocessing
import os
import requests
import shutil
from retry import retry

In [6]:
Map = geemap.Map()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=HdX9llGSgY8EiK4G_rC6KkAXy9Ed5JU5SzmGH8oAt4c&tc=b-mfoYz_6iFrxNoBVJV5NqS4MBMWw3DmrNM3_YZTg2o&cc=Sg1r7BzrUKqMpGZ0WM6NEb0SLtg-mARfybS8gCxUdmM

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ATx3LY7o0G0xU4ul5QxSFsl6OMR8W04YHZHDG4UIWQl0ttwcMIqXy8fZeac

Successfully saved authorization token.


In [13]:
ee.Initialize(opt_url="https://earthengine-highvolume.googleapis.com")

In [18]:
# image = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
#             .filterBounds(region) \
#             .filterDate('2021', '2022') \
#             .select('B8', 'B4', 'B3') \
#             .median() \
#             .visualize(min=0, max=4000) \
#             .clip(region)

image = ee.Image('users/tjm0042/PHD/GEDI_Height/stackedResampled');

In [24]:
# # region = Map.user_roi
# region = ee.Geometry.Polygon(
#     [
#         [
#             [-122.513695, 37.707998],
#             [-122.513695, 37.804359],
#             [-122.371902, 37.804359],
#             [-122.371902, 37.707998],
#             [-122.513695, 37.707998],
#         ]
#     ],
#     None,
#     False,
# )

region = image.geometry();

In [36]:
params = {
    "count": 100,  # How many image chips to export
    "buffer": 127,  # The buffer distance (m) around each point
    "scale": 100,  # The scale to do stratified sampling
    "seed": 1,  # A randomization seed to use for subsampling.
    "dimensions": "256x256",  # The dimension of each image chip
    "format": "GEO_TIFF" ,#"png",  # The output image format, can be png, jpg, ZIPPED_GEO_TIFF, GEO_TIFF, NPY
    "prefix": "tile_",  # The filename prefix
    "processes": 25,  # How many processes to used for parallel processing
    "out_dir": ".",  # The output directory. Default to the current working directly
}

In [37]:
Map.addLayer(image, {}, "Image")
Map.addLayer(region, {}, "ROI", False)
Map.centerObject(image, 10)
Map

Map(bottom=104929.0, center=[34.10155814460179, 72.70185391752715], controls=(WidgetControl(options=['position…

In [42]:
# def getRequests():
#     img = ee.Image(1).rename("Class").addBands(image)
#     points = img.stratifiedSample(
#         numPoints=params["count"],
#         region=region,
#         scale=params["scale"],
#         seed=params["seed"],
#         geometries=True,
#     )
#     Map.data = points
#     return points.aggregate_array(".geo").getInfo()


def getRequests():
    # img = ee.Image(1).rename("Class").addBands(image)
    points = GEDI_Training = ee.FeatureCollection('users/tjm0042/PHD/GEDI_Height/training_GEDI')

    # points = img.stratifiedSample(
    #     numPoints=params["count"],
    #     region=region,
    #     scale=params["scale"],
    #     seed=params["seed"],
    #     geometries=True,
    # )
    Map.data = points
    return points.aggregate_array(".geo").getInfo()

In [43]:
@retry(tries=10, delay=1, backoff=2)
def getResult(index, point):
    point = ee.Geometry.Point(point["coordinates"])
    region = point.buffer(params["buffer"]).bounds()

    if params["format"] in ["png", "jpg"]:
        url = image.getThumbURL(
            {
                "region": region,
                "dimensions": params["dimensions"],
                "format": params["format"],
            }
        )
    else:
        url = image.getDownloadURL(
            {
                "region": region,
                "dimensions": params["dimensions"],
                "format": params["format"],
            }
        )

    if params["format"] == "GEO_TIFF":
        ext = "tif"
    else:
        ext = params["format"]

    r = requests.get(url, stream=True)
    if r.status_code != 200:
        r.raise_for_status()

    out_dir = os.path.abspath(params["out_dir"])
    basename = str(index).zfill(len(str(params["count"])))
    filename = f"{out_dir}/{params['prefix']}{basename}.{ext}"
    with open(filename, "wb") as out_file:
        shutil.copyfileobj(r.raw, out_file)
    print("Done: ", basename)

In [44]:
%%time
logging.basicConfig()
items = getRequests()

pool = multiprocessing.Pool(params["processes"])
pool.starmap(getResult, enumerate(items))

pool.close()

Done:  180
Done:  330
Done:  030
Done:  165
Done:  015
Done: Done:  360
 240
Done:  150
Done:  210
Done: Done:   090255

Done:  195Done: Done: 
  105135

Done: Done: Done:    315060075


Done:  000
Done:  120
Done:  270
Done:  225
Done:  045
Done:  345
Done:  285
Done:  300
Done:  181
Done:  331
Done:  031
Done:  166
Done:  016
Done:  211
Done:  361
Done:  256
Done:  001
Done:  151
Done:  076
Done:  196
Done:  121
Done:  241
Done:  Done: 091
 Done:  286
046
Done: Done:   316
106
Done:  226Done: 
Done:   182301

Done:  061Done: 
 136
Done:  271
Done:  346
Done:  332
Done:  032
Done:  167
Done:  017
Done:  257
Done:  152
Done:  077
Done:  002
Done:  212
Done: Done:   047
122
Done:  227
Done:  062Done:  302

Done: Done:   092362

Done: Done:   107197Done: Done: 

  242183

Done:  287
Done:  347
Done:  137
Done:  272
Done:  333
Done:  317
Done:  033
Done:  Done:  168018

Done:  258
Done:  153
Done:  048
Done:  078
Done:  213
Done:  003
Done:  123
Done:  108
Done:  198
Done:  184
Done:  Don

In [45]:
Map.addLayer(Map.data, {}, "Sample points")
Map

Map(bottom=104929.0, center=[34.10155814460179, 72.70185391752715], controls=(WidgetControl(options=['position…